In [ ]:
!pip install transformers sentencepiece datasets

In [ ]:
pip install accelerate -U

In [ ]:
from transformers import AutoConfig, AutoModelForSequenceClassification

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
#bigscience/bloomz-560m
#bert-base-uncased
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


In [ ]:
from datasets import load_dataset

dataset = load_dataset("glue", "sst2")

In [ ]:
from transformers import AutoTokenizer
model_name = "bert-base-uncased"
#tokenizer = AutoTokenizer.from_pretrained(model_name)

#tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:

#from bertlt import  PrefixForSequenceClassification, PromptForSequenceClassification
from transformers import AutoConfig
config = AutoConfig.from_pretrained("Kowsher/L-Tuning")
config._name_or_path=model_name
config.hidden_size=768
config.num_hidden_layers=12
config.n_head=12
config.num_labels=2
config.pad_token_id=tokenizer.pad_token_id
config.hidden_dropout = 0.01
config.model_type='bert'
config.cls_token_id=tokenizer.cls_token_id
config.sep_token_id=tokenizer.sep_token_id
config.pooling=True

In [ ]:
b = AutoConfig.from_pretrained(model_name)
b

In [ ]:
from torch import nn
from transformers import DataCollatorForLanguageModeling
import torch
import random
from DataCollator import DataCollatorForLTCls
# define the label correctly from the dataset
label2prompt = {0: 'The text has negative sentiment, text: ',  1: 'The text has positive sentiment, text: '}

data_collator = DataCollatorForLTCls(config, tokenizer, label2prompt)




Prefix c (pre_seq_len) is 10


In [ ]:
from bertlt import PromptForSequenceClassification
config.pre_seq_len=10

model = PromptForSequenceClassification.from_pretrained(
    model_name,
    config=config,
)


Some weights of PromptForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['prompt_encoder.W.weight', 'prompt_encoder.W.bias', 'classifier.weight', 'prompt_encoder.projection.bias', 'classifier.bias', 'prompt_encoder.projection.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Total number of parameters in the model
total_parameters = model.num_parameters()

# Total number of trainable parameters in the model
trainable_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)

# Calculate the percentage of trainable parameters
percentage_trainable = (trainable_parameters / total_parameters) * 100

print(f"Total Parameters: {total_parameters}")
print(f"Trainable Parameters: {trainable_parameters}")
print(f"Percentage Trainable: {percentage_trainable:.20f}%")

Total Parameters: 109569027
Trainable Parameters: 86787
Percentage Trainable: 0.07920760307563924629%


In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import mean_squared_error
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback, IntervalStrategy
from sklearn.metrics import r2_score, accuracy_score, matthews_corrcoef
import numpy as np

def compute_metrics(p):
    logits = p.predictions
    preds = np.argmax(logits, axis=-1)
    labels = p.label_ids

    accuracy = accuracy_score(labels, preds)



    return {"acc": accuracy}

training_args = TrainingArguments(
    output_dir='./rfalcon_task',
    num_train_epochs=10,
    do_eval=True,
    #learning_rate=0.001,
    #bf16=True,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,

    logging_dir='./logs',
    logging_strategy="steps",
    logging_steps=100,
    evaluation_strategy="steps",
    eval_steps = 100,
    save_strategy="steps",
    save_steps=100,

    save_total_limit=2,
    load_best_model_at_end=True,
    #optim="paged_adamw_8bit",
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/kowsher/miniconda3/envs/LT/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/kowsher/miniconda3/envs/LT/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/home/kowsher/miniconda3/envs/LT/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/kowsher/miniconda3/envs/LT did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/kowsher/miniconda3/envs/LT/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('@/tmp/.ICE-unix/1423,unix/kowsher-XPS-8950'), PosixPath('local/kowsher-XPS-8950')}
  warn(msg)
/home/kowsher/miniconda3/envs/LT/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/etc/xdg/xdg-ubuntu')}
  warn(msg)
/home/kowsher/miniconda3/envs/LT/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found 

In [ ]:

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics, #compute_metrics1,#compute_metrics_classification,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=20)]
)

trainer.train()

Step,Training Loss,Validation Loss,Acc
100,0.299400,0.319096,0.873853
200,0.332400,0.303545,0.875000
300,0.302600,0.308947,0.877294
400,0.323600,0.296372,0.876147
500,0.299700,0.303932,0.869266
600,0.309800,0.303495,0.873853
700,0.329100,0.295724,0.873853
800,0.311100,0.301468,0.875000
900,0.326500,0.296935,0.876147
1000,0.303800,0.308704,0.873853


TrainOutput(global_step=3400, training_loss=0.28101540195240694, metrics={'train_runtime': 1094.0663, 'train_samples_per_second': 615.584, 'train_steps_per_second': 19.24, 'total_flos': 7163019700247808.0, 'train_loss': 0.28101540195240694, 'epoch': 1.62})